In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
import scipy
import glob
import sklearn 
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers, models, optimizers
from tensorflow.keras.layers import Input, Activation, Dense, LeakyReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras.optimizers import Adam, SGD
from keras_tuner import BayesianOptimization, HyperParameters
import os 

In [ ]:
# Data import
wt_filtered = ['wt_filtered_lcc_3_50.lccdata', 'wt_filtered_lcc_12_50.lccdata', 'wt_filtered_lcc_20_50.lccdata']

# filtered wt LCC data import
wt_f_var_names = ['wt_3f', 'wt_12f', 'wt_20f']

for var, file in zip(wt_f_var_names, wt_filtered):
    globals()[var] = pd.read_csv(file, sep='\t').drop(columns='Unnamed: 0')

# filtered mutant LCC data import
D132H_filtered = ['D132H_filtered_lcc_3_50.lccdata', 'D132H_filtered_lcc_12_50.lccdata', 'D132H_filtered_lcc_20_50.lccdata']
D132H_f_var_names = ['D132H_3f', 'D132H_12f', 'D132H_20f']

for var, file in zip(D132H_f_var_names, D132H_filtered):
    globals()[var] = pd.read_csv(file, sep='\t').drop(columns='Unnamed: 0')
    
# Visualization of dataset
print('WT for window size = 3')
display(wt_3f)
print('WT for window size = 12')
display(wt_12f)
print('WT for window size = 20')
display(wt_20f)

print('\n')
print('---------------------------------')
print('D132H for window size = 3')
display(D132H_3f)
print('D132H for window size = 12')
display(D132H_12f)
print('D132H for window size = 20')
display(D132H_20f)

In [ ]:
# Concateneate wt and mutant dataframes and rename columns

wt_f = pd.concat([wt_3f, wt_12f, wt_20f], axis = 1)
    
D132H_f = pd.concat([D132H_3f, D132H_12f, D132H_20f], axis = 1)
D132H_f.index = range(40000, 40000 + len(D132H_f)) # Modifying index of D132H so that there are no duplicate indices

colnames = [*range(0,12)]
colnames
wt_f.columns = colnames
D132H_f.columns = colnames

In [ ]:
def preprocessing(wt, mutant):
    
    wt_label = np.zeros(len(wt))  # Set wt labels to 0
    mutant_label = np.ones(len(mutant))  # Set mutant labels to 1

    # Create label dataframes with indices
    wt_label_df = pd.DataFrame({'class': wt_label})
    mutant_label_df = pd.DataFrame({'class': mutant_label}, index=range(40000, 40000 + len(mutant)))

    # Concatenate data frames and label dataframes
    X_train_full = pd.concat([wt, mutant])
    y_train_full_df = pd.concat([wt_label_df, mutant_label_df])

    # Normalize training data
    X_train_full = X_train_full.div(100)  # Adjust as necessary

    # Separate training and validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full_df, stratify=y_train_full_df['class'], test_size=0.2)

    print(X_train.shape)
    print(X_valid.shape)
    print(y_train.shape)
    print(y_valid.shape)

    return X_train, X_valid, y_train, y_valid

In [ ]:
X_train_f, X_valid_f, y_train_f, y_valid_f = preprocessing(wt_f, D132H_f)

In [ ]:
# Get autoencoder model
def get_ae(train_data, LeReLU_alpha=0.01):
    
    #Input layer
    input_layer = Input(shape=(train_data.shape[1]), name='ae_input')
    
    encoder = Dense(336, activation=LeakyReLU(alpha=LeReLU_alpha), name='e1')(input_layer)
    encoder = Dense(208, activation=LeakyReLU(alpha=LeReLU_alpha), name='e2')(encoder)
    encoder = Dense(240, activation=LeakyReLU(alpha=LeReLU_alpha), name='e3')(encoder)

    encoded = Dense(2, activation=LeakyReLU(alpha=LeReLU_alpha), name='ae_latent')(encoder)
    
    decoder = Dense(240, activation=LeakyReLU(alpha=LeReLU_alpha), name='d1')(encoded)
    decoder = Dense(208, activation=LeakyReLU(alpha=LeReLU_alpha), name='d2')(decoder)
    decoder = Dense(336, activation=LeakyReLU(alpha=LeReLU_alpha), name='d3')(decoder)

    output_layer = Dense(train_data.shape[1], activation=LeakyReLU(alpha=LeReLU_alpha), name='ae_output')(decoder)
    
    model = Model(input_layer, output_layer)

    return model

In [ ]:
# Get ae for filtered data
autoencoder = get_ae(X_train_f)

In [ ]:
# Print summary of ae model
autoencoder.summary()

In [ ]:
# Compile the model
autoencoder.compile(loss=tf.keras.losses.MeanSquaredError(), optimizer=tf.keras.optimizers.Adam(learning_rate = 0.00001))

In [ ]:
folder_name = "AET_CF_E37_Trial_1"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
name = "1_LSP_AET_37_CF_"

In [ ]:
X_train_f.to_csv(f'{folder_name}/X_train_f.csv')
y_train_f.to_csv(f'{folder_name}/y_train_f.csv')

X_valid_f.to_csv(f'{folder_name}/X_valid_f.csv')
y_valid_f.to_csv(f'{folder_name}/y_valid_f.csv')

In [ ]:
for counts in tqdm(range(38)):
    output_path = os.path.join(folder_name, name + f"Predictions{counts}.txt")
    with open(output_path, "w") as file:
        history = autoencoder.fit(X_train_f, X_train_f, batch_size=256, epochs=1000,
                                  validation_data=(X_valid_f, X_valid_f), verbose=1)

        training_history = pd.DataFrame(history.history)
        plt.plot(training_history)
        file_name_0 = os.path.join(folder_name, name + "Training_History" + str(counts))
        training_history.to_pickle(file_name_0)
        file_name_1 = os.path.join(folder_name, name + "Training_History" + str(counts) + ".png")
        plt.savefig(file_name_1, dpi=300)
        plt.clf()

        dr_model = tf.keras.models.Model(inputs=autoencoder.get_layer('ae_input').input,
                                         outputs=autoencoder.get_layer('ae_latent').output)
        dr_model.summary(print_fn=lambda x: file.write(x + '\n'))

        batch_size = 32
        x = []
        y = []
        z = []
        indices = []

        for batch_start in range(0, len(X_valid_f), batch_size):
            batch_end = min(batch_start + batch_size, len(X_valid_f))
            X_batch = np.array(X_valid_f.iloc[batch_start:batch_end])
            y_batch = y_valid_f.iloc[batch_start:batch_end]

            op_batch = dr_model.predict(X_batch, verbose=0)

            for i, op in enumerate(op_batch):
                z.append(y_batch.iloc[i]['class'])  # Access class label correctly
                x.append(op[0])
                y.append(op[1])
                indices.append(y_batch.iloc[i].name)  # Using .name to get the index of the row
                file.write(f"Prediction {batch_start + i}: {op}\n")

        df = pd.DataFrame()
        df['x'] = x
        df['y'] = y
        df['z'] = ["trajectory-" + str(k) for k in z]
        df['index'] = indices

        plt.figure(figsize=(8, 6))
        fig = sns.scatterplot(x='x', y='y', hue='z', data=df, s=10)
        file_name_2 = os.path.join(folder_name, name + 'Predictions' + str(counts) + ".png")
        fig.figure.savefig(file_name_2, dpi=300)
        plt.clf()

        file_name_3 = os.path.join(folder_name, name + 'Predictions' + str(counts))
        df.to_pickle(file_name_3)

        model_folder = os.path.join(folder_name, 'models')
        if not os.path.exists(model_folder):
            os.makedirs(model_folder)
        file_name = os.path.join(model_folder, 'saved_model_' + name + str(counts))
        autoencoder.save(file_name)